![](http://www.dataivy.cn/book/images/head.png)
<table align="center" bgcolor="#FFFFFF" border="0px">
   <tr bgcolor="#FFFFFF">
      <td><img src="http://www.dataivy.cn/book/images/release_date.svg"></td>
      <td><img src="http://www.dataivy.cn/book/images/python-3.7-green.svg"></td>
      <td><a href="http://www.dataivy.cn/blog/python_book_faq/"><img src="http://www.dataivy.cn/book/images/faq-visit_site-blue.svg"></a></td>
      <td><a href="http://www.dataivy.cn/blog/python_book_knows_issues/"><img src="http://www.dataivy.cn/book/images/known_issues.svg"></a></td>
   </tr>
</table>


<hr />

<div align="center"><h1> 第五章&nbsp;&nbsp;会员数据化运营</h1></div>

# 一、案例-基于RFM的精细化用户管理

## 说明

- 描述：“代码实操”以及内容延伸部分源代码
- 时间：2019-01-01
- 作者：宋天龙（Tony Song）
- 程序开发环境：win7 64位
- Python版本：64位 3.7
- 依赖库：time、numpy、pandas、sklearn、pyecharts
- 程序输入：sales.xlsx
- 程序输出：RFM得分数据写本地文件sales_rfm_score.xlsx和数据表（sales_rfm_score）

## 程序

### 导入库

In [1]:
import time  # 时间库

import numpy as np  # numpy库
import pandas as pd  # pandas库
import pymysql  # mysql连接库
from sklearn.ensemble import RandomForestClassifier # RF库

from pyecharts import Bar3D # 3D柱形图

### 读取数据

In [2]:
sheet_names = ['2015','2016','2017','2018','会员等级']
sheet_datas = [pd.read_excel('sales.xlsx',sheet_name=i) for i in sheet_names]

### 数据审查

In [3]:
for each_name,each_data in zip(sheet_names,sheet_datas):    
    print('[data summary for {0:=^50}]'.format(each_name))
    print('Overview:','\n',each_data.head(4))# 展示数据前4条
    print('DESC:','\n',each_data.describe())# 数据描述性信息
    print('NA records',each_data.isnull().any(axis=1).sum()) # 缺失值记录数    
    print('Dtypes',each_data.dtypes) # 数据类型

[data summary for =======================2015=======================]
Overview: 
           会员ID         订单号       提交日期    订单金额
0  15278002468  3000304681 2015-01-01   499.0
1  39236378972  3000305791 2015-01-01  2588.0
2  38722039578  3000641787 2015-01-01   498.0
3  11049640063  3000798913 2015-01-01  1572.0
DESC: 
                会员ID           订单号           订单金额
count  3.077400e+04  3.077400e+04   30774.000000
mean   2.918779e+10  4.020414e+09     960.991161
std    1.385333e+10  2.630510e+08    2068.107231
min    2.670000e+02  3.000305e+09       0.500000
25%    1.944122e+10  3.885510e+09      59.000000
50%    3.746545e+10  4.117491e+09     139.000000
75%    3.923593e+10  4.234882e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
NA records 0
Dtypes 会员ID             int64
订单号              int64
提交日期    datetime64[ns]
订单金额           float64
dtype: object
[data summary for =======================2016=======================]
Overview: 
           会员ID         订单号     

### 数据预处理

In [4]:
# 去除缺失值和异常值
for ind,each_data in enumerate(sheet_datas[:-1]):    
    sheet_datas[ind] = each_data.dropna()# 丢弃缺失值记录
    sheet_datas[ind] = each_data[each_data['订单金额'] > 1]# 丢弃订单金额<=1的记录
    sheet_datas[ind]['max_year_date'] = each_data['提交日期'].max() # 增加一列最大日期值

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
# 汇总所有数据
data_merge = pd.concat(sheet_datas[:-1],axis=0)
# 获取各自年份数据
data_merge['date_interval'] = data_merge['max_year_date']-data_merge['提交日期']
data_merge['year'] = data_merge['提交日期'].dt.year
# 转换日期间隔为数字
data_merge['date_interval'] = data_merge['date_interval'].apply(lambda x: x.days) # 转换日期间隔为数字
#data_merge.head()

In [6]:
# 按会员ID做汇总
rfm_gb = data_merge.groupby(['year','会员ID'],as_index=False).agg({'date_interval': 'min',  # 计算最近一次订单时间
                                                   '提交日期': 'count', # 计算订单频率
                                                   '订单金额': 'sum'})  # 计算订单总金额
# 重命名列名
rfm_gb.columns =  ['year','会员ID','r','f','m']
rfm_gb.head()

,year,会员ID,r,f,m
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0


### 确定RFM划分区间

In [7]:
# 查看数据分布
desc_pd = rfm_gb.iloc[:,2:].describe().T
print(desc_pd)
# 定义区间边界
r_bins = [-1,79,255,365] # 注意起始边界小于最小值
f_bins = [0,2,5,130] 
m_bins = [0,69,1199,206252]

      count         mean          std  min   25%    50%     75%       max
r  148591.0   165.524043   101.988472  0.0  79.0  156.0   255.0     365.0
f  148591.0     1.365002     2.626953  1.0   1.0    1.0     1.0     130.0
m  148591.0  1323.741329  3753.906883  1.5  69.0  189.0  1199.0  206251.8


### 计算RFM因子权重

In [8]:
# 匹配会员等级和rfm得分
rfm_merge = pd.merge(rfm_gb,sheet_datas[-1],on='会员ID',how='inner')

In [9]:
# rf获得rfm因子得分
clf = RandomForestClassifier()
clf = clf.fit(rfm_merge[['r','f','m']],rfm_merge['会员等级'])
weights = clf.feature_importances_
print('feature importance:',weights)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


feature importance: [0.39042291 0.00670539 0.6028717 ]


### RFM计算过程

In [10]:
# RFM分箱得分
rfm_gb['r_score'] = pd.cut(rfm_gb['r'], r_bins, labels=[i for i in range(len(r_bins)-1,0,-1)])  # 计算R得分
rfm_gb['f_score'] = pd.cut(rfm_gb['f'], f_bins, labels=[i+1 for i in range(len(f_bins)-1)])  # 计算F得分
rfm_gb['m_score'] = pd.cut(rfm_gb['m'], m_bins, labels=[i+1 for i in range(len(m_bins)-1)])  # 计算M得分

In [11]:
# 计算RFM总得分
# 方法一：加权得分
rfm_gb = rfm_gb.apply(np.int32) # cate转数值
rfm_gb['rfm_score'] = rfm_gb['r_score'] * weights[0] + rfm_gb['f_score'] * weights[1] + rfm_gb[
'm_score'] * weights[2]

In [12]:
# 方法二：RFM组合
rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
rfm_gb['m_score'] = rfm_gb['m_score'].astype(np.str)
rfm_gb['rfm_group'] = rfm_gb['r_score'].str.cat(rfm_gb['f_score']).str.cat(
rfm_gb['m_score'])

### 保存RFM结果到Excel

In [13]:
rfm_gb.to_excel('sales_rfm_score.xlsx')  # 保存数据为Excel

### 写数据到数据库

In [14]:
# 数据库信息
config = {'host': '127.0.0.1',  # 默认127.0.0.1
          'user': 'root',  # 用户名
          'password': '123456',  # 密码
          'port': 3306,  # 端口，默认为3306
          'database': 'python_data',  # 数据库名称
          'charset': 'gb2312'  # 字符编码
          }

In [15]:
# 建表操作
con = pymysql.connect(**config)  # 建立mysql连接
cursor = con.cursor()  # 获得游标
cursor.execute("show tables")  # 查询表
table_list = [t[0] for t in cursor.fetchall()]  # 读出所有库
# 查找数据库是否存在目标表，如果没有则新建
table_name = 'sales_rfm_score'  # 要写库的表名
if not table_name in table_list:  # 如果目标表没有创建
    cursor.execute('''
    CREATE TABLE %s (
    userid               VARCHAR(20),
    r_score               int(2),
    f_score              int(2),
    m_score              int(2),
    rfm_score              DECIMAL(10,2),
    rfm_group              VARCHAR(10),
    insert_date              VARCHAR(20)
    )ENGINE=InnoDB DEFAULT CHARSET=gb2312
    ''' % table_name)  # 创建新表

In [16]:
# 梳理数据
write_db_data = rfm_gb[['会员ID','r_score','f_score','m_score','rfm_score','rfm_group']] # 主要数据
timestamp = time.strftime('%Y-%m-%d', time.localtime(time.time()))  # 日期

In [17]:
# 写库
for each_value in write_db_data.values:
    insert_sql = "INSERT INTO `%s` VALUES ('%s',%s,%s,%s,%s,'%s','%s')" % \
                 (table_name, each_value[0], each_value[1], each_value[2], \
                  each_value[3],each_value[4],each_value[5],
                  timestamp)  # 写库SQL依据
    cursor.execute(insert_sql)  # 执行SQL语句，execute函数里面要用双引号
    con.commit()  # 提交命令
cursor.close()  # 关闭游标
con.close()  # 关闭数据库连接

C:\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'rfm_score' at row 1")
  result = self._query(query)


### RFM图形展示

In [18]:
# 图形数据汇总
display_data = rfm_gb.groupby(['rfm_group','year'],as_index=False)['会员ID'].count()
display_data.columns = ['rfm_group','year','number']
display_data['rfm_group'] = display_data['rfm_group'].astype(np.int32)
display_data.head()

,rfm_group,year,number
0,111,2015,2180
1,111,2016,1498
2,111,2017,3169
3,111,2018,2271
4,112,2015,3811


In [19]:
# 显示图形
bar3d = Bar3D("", width=900, height=600)
range_color = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf',
               '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
bar3d.add(
    "rfm分组结果",
    "",
    "",
    [d.tolist() for d in display_data.values],
    is_visualmap=True,
    visual_range=[0, display_data['number'].max()],
    visual_range_color=range_color,
    grid3d_width=200,
    grid3d_height=80,
    grid3d_depth=80
)
bar3d

# 二、案例-基于嵌套Pipeline和FeatureUnion复合数据工作流的营销响应预测

## 说明

- 描述：“代码实操”以及内容延伸部分源代码
- 时间：2019-01-01
- 作者：宋天龙（Tony Song）
- 程序开发环境：win7 64位
- Python版本：64位 3.7
- 依赖库：time、numpy、pandas、sklearn、imblearn
- 程序输入：order.xlsx
- 程序输出：预测数据写本地文件order_predict_result.xlsx

## 程序

### 导入库

In [1]:
import time

import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import f1_score,accuracy_score,precision_score
from imblearn.over_sampling import SMOTE  # 过抽样处理库SMOTE

### 读取数据

In [2]:
raw_data = pd.read_excel('order.xlsx', sheet_name=0)  # 读出Excel的第一个sheet

### 数据审查

In [3]:
# 查看基本状态
print('records:{0} features:{1}'.format(raw_data.shape[0], (raw_data.shape[1] - 1)))  # 打印数据集形状
# 查看缺失值
print('NaN records count:',raw_data.isnull().any(axis=1).count())  # 具有缺失值的记录数
na_cols = raw_data.isnull().any()
print('NaN cols',na_cols[na_cols.values==True]) # 输出缺失值的列
# 查看样本均衡情况
print('samples distribution:',raw_data['value_level'].groupby(raw_data['response']).count())

records:39999 features:113
NaN records count: 39999
NaN cols age                True
total_pageviews    True
edu                True
edu_ages           True
user_level         True
industry           True
act_level          True
sex                True
red_money          True
region             True
dtype: bool
samples distribution: response
0    30415
1     9584
Name: value_level, dtype: int64


### 数据预处理

In [4]:
# Nan处理
na_rules = {'age': raw_data['age'].mean(),
            'total_pageviews': raw_data['total_pageviews'].mean(),
            'edu': raw_data['edu'].median(),
            'edu_ages': raw_data['edu_ages'].median(),
            'user_level': raw_data['user_level'].median(),
            'industry': raw_data['user_level'].median(),
            'act_level': raw_data['act_level'].median(),
            'sex': raw_data['sex'].median(),
            'red_money': raw_data['red_money'].mean(),
            'region': raw_data['region'].median()
            }  # 字典：定义各个列数据填充方法
raw_data = raw_data.fillna(na_rules)  # 使用指定方法填充缺失值
print('Check NA exists:', raw_data.isnull().any().sum())  # 查找是否还有缺失值

Check NA exists: 0


In [5]:
# 分割特征和目标
num = int(0.7*raw_data.shape[0])
X,y = raw_data.drop('response', axis=1),raw_data['response']
X_train,X_test = X.iloc[:num,:],X.iloc[num:,:]
y_train,y_test = y.iloc[:num],y.iloc[num:]

In [6]:
# 样本均衡
model_smote = SMOTE()  # 建立SMOTE模型对象
x_smote_resampled, y_smote_resampled = model_smote.fit_sample(X_train, y_train)  # 输入数据并作过抽样处理

### 模型训练

In [7]:
# 建立pipeline中用到的模型对象
model_etc = ExtraTreesClassifier()  # ExtraTree，用于EFE的模型对象
model_rfe = RFE(model_etc)  # 使用RFE方法提取重要特征
model_lda = LinearDiscriminantAnalysis()  # LDA模型对象
model_rf = RandomForestClassifier()  # 分类对象

In [8]:
# 构建带有嵌套的pipeline
pipelines = Pipeline([
    ('feature_union', FeatureUnion(  # 组合特征pipeline
        transformer_list=[
            ('model_rfe', model_rfe),  # 通过RFE中提取特征
            ('model_lda', model_lda),  # 通过LDA提取特征
        ],
        transformer_weights={  # 建立不同特征模型的权重
            'model_rfe': 1,  # RFE模型权重
            'model_lda': 0.8,  # LDA模型权重
        },
    )),
    ('model_rf', model_rf),  # rf模型对象
])

In [9]:
# 设置pipe参数值
pipelines.set_params(feature_union__model_rfe__estimator__n_estimators=20)  # ExtraTreesClassifier中n_estimators值
pipelines.set_params(feature_union__model_rfe__estimator__n_jobs=-1)  # ExtraTreesClassifier中n_jobs值
pipelines.set_params(feature_union__model_rfe__n_features_to_select=20)  # RFE中n_features_to_select值
pipelines.set_params(feature_union__model_lda__n_components=1)  # LDA中n_components值
pipelines.set_params(feature_union__n_jobs=-1)  # FeatureUnion中n_jobs值
# project_pipeline.get_params()  # 打印pipline参数详情

Pipeline(memory=None,
     steps=[('feature_union', FeatureUnion(n_jobs=-1,
       transformer_list=[('model_rfe', RFE(estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_spl...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [10]:
# pipeline交叉检验
cv = StratifiedKFold(3)  # 设置交叉检验
score_list = list()  # 建立空列表，用于存放交叉检验得分
time_list = list()  # 建立空列表，用于存储时间
n_estimators = [10, 50, 100]  # 设置pipeline中rf的n_estimators值域
for parameter in n_estimators:  # 遍历每个参数值
    t1 = time.time()  # 记录交叉检验开始的时间
    print('set parameters: %s' % parameter)  # 打印当前模型使用的参数
    pipelines.set_params(model_rf__n_estimators=parameter)  # 通过管道设置分类模型参数
    score_tmp = cross_val_score(pipelines, X_train, y_train, scoring='accuracy',cv=cv,n_jobs=-1)  # 使用交叉检验计算得分
    time_list.append(time.time() - t1)  # 计算交叉检验时间并追加到列表
    score_list.append(score_tmp)  # 将得分追加到列表

set parameters: 10
set parameters: 50
set parameters: 100


In [11]:
# 组合交叉检验得分和详情数据
time_pd = pd.DataFrame.from_dict({'n_estimators':n_estimators,'time':time_list})
score_pd = pd.DataFrame(score_list,columns=[''.join(['score', str(i+1)]) for i in range(len(score_list))])
pd_merge = pd.concat((time_pd,score_pd),axis=1) # 完整数据框
pd_merge['score_mean'] = pd_merge.iloc[:, 2:-1].mean(axis=1)  # 计算得分均值
pd_merge['score_std'] = pd_merge.iloc[:, 2:-2].std(axis=1)  # 计算得分std
print(pd_merge.head())

   n_estimators    time    score1    score2    score3  score_mean  score_std
0            10  70.203  0.879794  0.875603  0.878804    0.877699   0.002964
1            50  68.001  0.888579  0.887496  0.887913    0.888038   0.000766
2           100  69.018  0.887508  0.891889  0.887484    0.889699   0.003098


In [12]:
# 将最优参数设置到模型中，并训练pipeline
pipelines.set_params(model_rf__n_estimators=50)  # 设置最优参数值
pipelines.fit(X_train, y_train)  # 训练pipeline模型

Pipeline(memory=None,
     steps=[('feature_union', FeatureUnion(n_jobs=-1,
       transformer_list=[('model_rfe', RFE(estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_spl...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 模型效果检验

In [13]:
# 模型效果检验
pre_test = pipelines.predict(X_test)
scores = [i(y_test,pre_test) for i in [f1_score,accuracy_score,precision_score]]
print('socres result: f1:{0},accuracy:{1},precision:{2}'.format(scores[0],scores[1],scores[2]))

socres result: f1:0.7611671469740635,accuracy:0.8895,precision:0.786378861183476


### 预测新数据集

In [14]:
# 读取新数据集
new_data = pd.read_excel('order.xlsx', sheet_name=1)  # 读取要预测的数据集

In [15]:
# 查看基本状态
print('records:{0} features:{1}'.format(new_data.shape[0], (new_data.shape[1])))  # 打印数据集形状
# 缺失值处理
print('NaN records count:',new_data.isnull().any(axis=1).count())  # 具有缺失值的记录数

records:8843 features:113
NaN records count: 8843


In [16]:
# 填充NA值
new_data_fillna = new_data.fillna(na_rules)  # 使用指定方法填充缺失值

In [17]:
# 预测概率
pre_labels = pd.DataFrame(pipelines.predict(new_data_fillna), columns=['labels'])  # 获得预测标签
pre_pro = pd.DataFrame(pipelines.predict_proba(new_data_fillna), columns=['pro1', 'pro2'])  # 获得预测概率
predict_pd = pd.concat((pre_labels, pre_pro), axis=1)  # 合并数据
print(predict_pd.head())  # 打印前2条结果

   labels  pro1  pro2
0       0  0.88  0.12
1       0  0.96  0.04
2       0  0.96  0.04
3       0  1.00  0.00
4       0  0.90  0.10


### 保存到文件

In [18]:
writer = pd.ExcelWriter('order_predict_result.xlsx')  # 创建写入文件对象
predict_pd.to_excel(writer, 'Sheet1')  # 将数据写入sheet1
writer.save()
writer.close()